In [ ]:
# Count time of process
import time
start = time.time()

<b> IMPORT PACKAGE

In [ ]:
import requests
import numpy as np
import pandas as pd
import urllib.request
import os
import time
from datetime import datetime, date, timedelta
from bs4 import BeautifulSoup
from random import randint
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait as wait
from selenium.webdriver.support import expected_conditions as EC
from tqdm import tqdm

<b> CONNECT TO BIGQUERY

<b> CONNECT TO URL (SELENIUM)

In [ ]:
#Chrome Settings
chrome_options = webdriver.chrome.options.Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

In [ ]:
#Get URL
driver = webdriver.Chrome('chromedriver',options=chrome_options)
driver.get("https://www.bi.go.id/id/statistik/informasi-kurs/transaksi-bi/Default.aspx")

<b> LIST OF THE CURRENCY NAME

In [ ]:
### Defined currency list name ###
list_curr = ['AUD  ','CNY  ','EUR  ','JPY  ','MYR  ','SGD  ','THB  ','USD  ']
list_curr_cleaned = list(map(str.strip, list_curr))
print(list_curr_cleaned)

['AUD', 'CNY', 'EUR', 'JPY', 'MYR', 'SGD', 'THB', 'USD']


<b> SET CURRENT DATE

In [ ]:
month = ['Jan','Feb','Mar','Apr','Mei','Jun','Jul','Ags','Sep','Okt','Nov','Des']

In [ ]:
# get date range
currentdate = pd.to_datetime(datetime.today(), format='%d %m %Y')

target_month_year_string = currentdate.strftime("%d-%m-%Y")
print(target_month_year_string)

14-10-2023


<b> CRAWLING PROCESS

In [ ]:
### SET TIME SERIES MENU  ###
wait(driver, 20).until(EC.visibility_of_element_located((By.XPATH, "//select[@id='selectPeriod']/option[@value='harian']"))).click()

In [ ]:
### DATE PICKER ###
# start date
startfield = wait(driver, 20).until(EC.visibility_of_element_located((By.XPATH, "//input[@id='ctl00_PlaceHolderMain_g_6c89d4ad_107f_437d_bd54_8fda17b556bf_ctl00_txtTanggal']")))
driver.execute_script("arguments[0].value = '"+target_month_year_string+"';", startfield)

# search button
button_search = wait(driver, 20).until(EC.visibility_of_element_located((By.ID, "ctl00_PlaceHolderMain_g_6c89d4ad_107f_437d_bd54_8fda17b556bf_ctl00_btnSearch2")))
driver.execute_script("arguments[0].click();", button_search)
print("Success click the button search!")

Success click the button search!


In [ ]:
### RAW DATA ###
soup = BeautifulSoup(driver.page_source, "html.parser")
table = soup.find('table',{'class':'table table-striped table-no-bordered table-lg'})
rows = table.findAll('tr')

In [ ]:
table

<table class="table table-striped table-no-bordered table-lg">
<thead class="thead color-blue">
<tr class="table-header">
<th class="text-right" scope="col">
                                Mata Uang</th>
<th class="text-right" scope="col">
                                Nilai</th>
<th class="text-right" scope="col">
                                Kurs Jual</th>
<th class="text-right" scope="col">
                                Kurs Beli</th>
<th class="text-right" scope="col">
                                Grafik</th>
</tr>
</thead>
<tbody>
<tr>
<td class="text-right">AUD  </td>
<td class="text-right">1</td>
<td class="text-right">10.113,73</td>
<td class="text-right">10.009,97</td>
<td class="text-right">
<input alternatetext="graph" autopostback="True" class="btn" id="ctl00_PlaceHolderMain_g_6c89d4ad_107f_437d_bd54_8fda17b556bf_ctl00_ListViewKurs_ctrl0_btnGraphImage" name="ctl00$PlaceHolderMain$g_6c89d4ad_107f_437d_bd54_8fda17b556bf$ctl00$ListViewKurs$ctrl0$btnGraphImage" oncli

In [ ]:
#Close driver
driver.close()

In [ ]:
type(rows)

bs4.element.ResultSet

In [ ]:
### CONVERT TO DATAFRAME ###
result = {'currency':[],
          'amount':[],
          'sell_rate':[],
          'buy_rate':[]}

for row in rows[1:]:
    cols = row.findAll('td')
    result['currency'].append(cols[0].text.lstrip().rstrip())
    result['amount'].append(cols[1].text.lstrip().rstrip())
    result['sell_rate'].append(cols[2].text.lstrip().rstrip())
    result['buy_rate'].append(cols[3].text.lstrip().rstrip())

result = pd.DataFrame(result)
result['date'] = currentdate.strftime('%Y-%m-%d')
result.tail()

,currency,amount,sell_rate,buy_rate,date
20,SEK,1,"1.446,88","1.432,09",2023-10-14
21,SGD,1,"11.583,73","11.467,62",2023-10-14
22,THB,1,"436,17","431,71",2023-10-14
23,USD,1,"15.780,51","15.623,49",2023-10-14
24,VND,1,"0,65","0,64",2023-10-14


In [ ]:
### CONVERT TO TABULAR FORMAT ###
result_tab = result.melt(id_vars=["currency","date","amount"], var_name="category")

In [ ]:
# sort columns name
result_tab = result_tab[['date','currency','amount','value','category']]

In [ ]:
result_tab['currency'].unique()

array(['AUD', 'BND', 'CAD', 'CHF', 'CNH', 'CNY', 'DKK', 'EUR', 'GBP',
       'HKD', 'JPY', 'KRW', 'KWD', 'LAK', 'MYR', 'NOK', 'NZD', 'PGK',
       'PHP', 'SAR', 'SEK', 'SGD', 'THB', 'USD', 'VND'], dtype=object)

In [ ]:
result_tab

,date,currency,amount,value,category
0,2023-10-14,AUD,1,"10.113,73",sell_rate
1,2023-10-14,BND,1,"11.583,73",sell_rate
2,2023-10-14,CAD,1,"11.611,85",sell_rate
3,2023-10-14,CHF,1,"17.510,55",sell_rate
4,2023-10-14,CNH,1,"2.162,75",sell_rate
5,2023-10-14,CNY,1,"2.162,10",sell_rate
6,2023-10-14,DKK,1,"2.247,78",sell_rate
7,2023-10-14,EUR,1,"16.763,64",sell_rate
8,2023-10-14,GBP,1,"19.405,29",sell_rate
9,2023-10-14,HKD,1,"2.017,76",sell_rate
